<a href="https://colab.research.google.com/github/Tasneem-Ibrahim/Python/blob/main/Project_2_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2 - RAG
LangChain RAG with Google Gemini Flash and Pinecone

### Step 1: Set Up Environment Variables

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai google-generativeai openai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00


### Step 2: Import Libraries and Initialize Pinecone

In [ ]:

#import getpass
#import os
#import time

from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)

### Step 3: Create or Connect to Pinecone Index

In [ ]:
import time

index_name = "online-rag-project"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
      name=index_name,
      dimension=768,
      metric="cosine",
      spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
       time.sleep(1)
index = pc.Index(index_name)

### Step 4: Initialize Google Generative AI Embeddings

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

### Step 5: Generate Query Embedding

In [ ]:
vector = embeddings.embed_query("Building a system leveraging RAG for better responses. ")
vector[:5]

[0.03307317569851875,
 -0.05718652531504631,
 -0.013254886493086815,
 0.029134679585695267,
 0.002929301466792822]

### Step 6: Create Pinecone Vector Store

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

### Step 7: Prepare Documents for Vector Store

In [ ]:
# Data Save

from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [ ]:
len(documents)

10

### Step 8: Generate UUIDs and Add Documents

In [ ]:
from uuid import uuid4
uuid4()


UUID('535d3031-5dba-46c5-86a0-b1643b9fbe17')

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

# Delete items from vector store:
# vector_store.delete(ids=[uuids[-1]])

['9f58d6ea-88fb-4c50-9594-4ac212fc8346',
 '47d68710-f5af-41b2-9364-ad9667f025d1',
 '9b0ab6bb-9e16-4f72-bee5-a1ed53bcbc6f',
 '0b22f3c3-1882-47da-851b-1f6d81c99276',
 '075326bc-9b7e-4723-bd33-8991dff11b64',
 '182cb806-3eac-4e7a-9b89-2bf075a10e71',
 '40320eca-1beb-4422-af82-f1af7bff1957',
 'b1aeca40-e34a-4324-a389-21c307225788',
 '4aaf8116-b47d-4958-acaf-5dd67963f726',
 '977cb7e0-48e9-44da-853d-8c58cba99546']

### Step 9: Perform Similarity Search

In [ ]:
# Data Reterive
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=5,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.668031] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


#### Query by turning into retriever

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='20eedb21-2968-483d-9520-43eaf3350854', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

### Step 10: Configuring Google's Gemini Model for Conversational AI

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

### Step 11: Function to Generate User-Specific Answers Using Vector Search and LLM

In [ ]:
def answer_to_user(query: str):

    vector_results = vector_store.similarity_search(query, k=2)
    print(len(vector_results ))

    # ToDo pass to vector model result + User query
    # Final answer by invoking the LLM
    final_answer = llm.invoke(f"ANSWER THIS USER QUERY: {query} Here are some references to answer:\n{vector_results}")

    return final_answer


#### Executing User Query and Retrieving Final Answer from LLM

In [ ]:
answer = answer_to_user("LangGraph is the best framework for building stateful, agentic applications!")

2


In [ ]:
answer.content

'The provided text only contains two tweets stating that LangGraph is the best framework for building stateful, agentic applications.  There is no further information or evidence to support this claim.  Therefore, the assertion that LangGraph is the *best* framework is unsubstantiated.  While the tweets express a strong opinion,  more information and comparative analysis with other frameworks would be needed to validate this statement.'